In [ ]:
'''
문제 3-1 :  콘텐츠분쟁해결 RAG 시스템 - 간단 실습 가이드
콘텐츠분쟁해결 사례집을 활용한 법률 자문 RAG 시스템을 순차적으로 구축하는 실습입니다.

1단계: 문서 분할 설정 
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,        #  법률 사례는 1200-1800자 권장
    chunk_overlap=300,      #  사례 맥락 보존을 위해 200-400자
    separators=[
        "\n【사건개요】",     #  법률 문서 섹션 구분자
        "\n【쟁점사항】",     #  쟁점 부분 구분
        "\n【처리경위】",     #  처리 과정 구분
        "\n【처리결과】",     #  결과 부분 구분
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
2단계: 임베딩 모델 설정 
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",  #  한국어 법률 용어에 최적
    dimensions=1536                  #  고성능 임베딩 차원
)
3단계: 검색기 설정 
pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        #  또는 "mmr" (다양성 고려시)
    search_kwargs={"k": 5}          #  상위 5개 관련 사례 검색
)
4단계: LLM 설정 
pythonllm = ChatOpenAI(
    model="gpt-4o",                 #  또는 "gpt-4o-mini" (경제적)
    temperature=0.2,                #  법률 조언은 정확성 중시
    max_tokens=2000                 #  충분한 답변 길이
)
5단계: 법률 자문 프롬프트 작성 
pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""

6단계: QA 체인 생성 
pythonqa_chain = RetrievalQA.from_chain_type(
    llm=llm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=retriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": prompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)
7단계: 테스트 질문 작성 
pythontest_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

8단계: 분쟁 유형 분류 함수 ( 선택 )
def classify_dispute_type(query):
    game_keywords = ["게임", "아이템", "계정", "캐릭터", "레벨", "길드", "온라인게임"]
    elearning_keywords = ["강의", "온라인교육", "이러닝", "수강", "환불", "화상교육"]
    web_keywords = ["웹사이트", "무료체험", "자동결제", "구독", "사이트"]
    
    query_lower = query.lower()
    
    if any(keyword in query_lower for keyword in game_keywords):
        return "게임"
    elif any(keyword in query_lower for keyword in elearning_keywords):
        return "이러닝"
    elif any(keyword in query_lower for keyword in web_keywords):
        return "웹콘텐츠"
    else:
        return "기타"


프롬프트 기본 템플릿:
python# 기본 법률 자문 프롬프트 (5분 만에 완성)
prompt_template = """당신은 콘텐츠 분야 전문 법률 자문사입니다.

관련 분쟁사례: {context}
상담 내용: {question}

답변 가이드라인:
1. 사례를 근거로 답변하세요
2. 관련 법령을 명시하세요  
3. 단계별 해결방안을 제시하세요
4. 유사 사례를 참조하세요
5. 없는 정보는 "확인할 수 없습니다"라고 하세요

'''

In [6]:


from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv
import os


print(" === base 작업 === ")
load_dotenv()

# 문서 로드
loader = PyPDFLoader("./data/콘텐츠분쟁해결_사례.pdf")

# 문자열 분할 
pythontext_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,       
    chunk_overlap=300,      
    separators=[
        "\n【사건개요】",   
        "\n【쟁점사항】",   
        "\n【처리경위】",  
        "\n【처리결과】",    
        "\n■", "\n\n", "\n", ".", " ", ""
    ]
)
chunks = pythontext_splitter.split_documents(loader.load())

# 임베드 ai 설정
pythonembeddings = OpenAIEmbeddings(
    model="text-embedding-3-large", 
    dimensions=1536                  
)

# 벡터 설정
vectorstore = FAISS.from_documents(chunks, pythonembeddings)
pythonretriever = vectorstore.as_retriever(
    search_type="similarity",        
    search_kwargs={"k": 5}         
)

# ai 설정
pythonllm = ChatOpenAI(
    model="gpt-4o",                
    temperature=0.2,               
    max_tokens=2000                 
)
print(" === base 작업 완료 === ")

 === base 작업 === 
 === base 작업 완료 === 


In [9]:

# 법률 자문 전용 프롬프트 설정
pythonprompt_template = """
당신은 콘텐츠 분야 전문 법률 자문사입니다. 
아래 분쟁조정 사례들을 바탕으로 정확하고 전문적인 법률 조언을 제공해주세요.

관련 분쟁사례:
{context}

상담 내용: {question}

답변 가이드라인:
1. 제시된 사례들을 근거로 답변하세요                    # 사례 기반 답변
2. 관련 법령이나 조항이 있다면 명시하세요               # 법적 근거 제시
3. 비슷한 사례의 처리경위와 결과를 참고하여 설명하세요    # 판례 참조
4. 실무적 해결방안을 단계별로 제시하세요               #  실무 가이드
5. 사례에 없는 내용은 "제시된 사례집에서는 확인할 수 없습니다"라고 명시하세요  #  한계 인정

전문 법률 조언:"""

legal_prompt = PromptTemplate(
    template= pythonprompt_template,
    input_variables=["context", "question"]
)

legal_qa_chain = RetrievalQA.from_chain_type(
    llm=pythonllm,                        #  위에서 설정한 LLM
    chain_type="stuff",             #  문서들을 하나로 합쳐서 처리
    retriever=pythonretriever,            #  위에서 설정한 검색기
    chain_type_kwargs={"prompt": legal_prompt},  #  법률 자문 프롬프트
    return_source_documents=True    #  참조 문서도 함께 반환
)

legal_questions = [
    "온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?",
    "인터넷 강의를 중도 해지하려고 하는데 과도한 위약금을 요구받고 있습니다. 정당한가요?",
    "무료체험 후 자동으로 유료전환되어 요금이 청구되었습니다. 환불 가능한가요?",
    "미성년자가 부모 동의 없이 게임 아이템을 구매했습니다. 환불받을 수 있는 방법이 있나요?",
    "온라인 교육 서비스가 광고와 다르게 제공되어 계약을 해지하고 싶습니다. 가능한가요?"
]

for question in legal_questions:
    print(f"\n\n")
    print(f"상담내용 : {question}")
    
    result = legal_qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    print(f"\n 법률 자문:")
    print(answer)



상담내용 : 온라인 게임에서 시스템 오류로 아이템이 사라졌는데, 게임회사가 복구를 거부하고 있습니다. 어떻게 해결할 수 있나요?

 법률 자문:
온라인 게임에서 시스템 오류로 인해 아이템이 사라졌을 경우, 게임회사가 복구를 거부하는 상황에 대한 법률적 조언을 드리겠습니다.

1. **사례 기반 답변:**
   - 제시된 사례 중 2006년과 2009년의 시스템 오류로 인한 아이템 소실 사례를 참고할 수 있습니다. 두 사례 모두 시스템 오류로 인한 아이템 소실에 대해 복구를 요구했으나, 게임 회사는 계정 명의 문제나 시스템 오류의 부재를 이유로 복구를 거부했습니다.

2. **법적 근거 제시:**
   - 일반적으로 온라인 게임의 이용약관에는 시스템 오류로 인한 손실에 대한 책임을 제한하는 조항이 포함되어 있습니다. 이러한 조항은 계약법에 따라 유효할 수 있으나, 소비자 보호법에 따라 불공정한 약관으로 판단될 가능성도 있습니다.

3. **판례 참조:**
   - 2009년 사례에서는 시스템 오류로 인한 손실이 인정되지 않았고, 게임 회사는 시스템 오류가 없었다고 주장했습니다. 이는 시스템 오류가 실제로 발생했는지를 입증하는 것이 중요함을 시사합니다.

4. **실무 가이드:**
   - **증거 수집:** 시스템 오류가 발생했음을 입증할 수 있는 증거(스크린샷, 로그 기록 등)를 수집하세요.
   - **공식적인 이의 제기:** 게임 회사의 고객 서비스나 공식 채널을 통해 이의 제기를 하세요. 이때, 수집한 증거를 함께 제출하세요.
   - **소비자 보호 기관에 문의:** 한국소비자원과 같은 소비자 보호 기관에 상담을 요청하고, 중재를 요청할 수 있습니다.
   - **법적 조치 고려:** 만약 위의 방법으로 해결되지 않는다면, 법적 조치를 고려할 수 있습니다. 이 경우, 변호사와 상담하여 소송 가능성을 검토하세요.

5. **한계 인정:**
   - 제시된 사례집에서는 시스템 오류로 인한 아이템 소실에 대해 게임 회사가 자발적으로 복구한 사례는 확인할 수 